## Query for a Demo Site
Start by querying a site for a patient with known session data (i.e. Dose_Hst records).  We will fetch the first site for the patient and store the SIT_ID and SIT_SET_ID for subsequent queries.

In [ ]:
from pymedphys import mosaiq
msq_server, test_db_name, pat_id1 = '.', 'MosaiqTest94086', 10003

connection = mosaiq.connect(msq_server, database=test_db_name)
sites = mosaiq.execute(
    connection,
    """
    SELECT 
        SIT_ID, 
        SIT_SET_ID, 
        Site_Name,
        Notes
    FROM Site 
    WHERE 
        Version = 0 
        AND Pat_ID1 = %(pat_id1)s
    """,
    { "pat_id1": pat_id1 })

sit_id, sit_set_id = sites[0][0], sites[0][1]
print(f"SIT_ID:{sit_id}  SIT_SET_ID:{sit_set_id}  Site_Name:{sites[0][2]}  Notes:{sites[0][3]}")

## Query for Dose_Hst
For the select site, query for Dose_Hst records that are associated via SIT_ID.  List the first ten of the Dose_Hst records.

Dose_Hst is not a versioned entity, so no need to get the tip versions.

In [ ]:
import pprint as pprint

dose_hsts = mosaiq.execute(
    connection,
    """
    SELECT 
        Tx_DtTm
    FROM Dose_Hst 
    WHERE 
        Dose_Hst.SIT_ID = %(sit_id)s
    ORDER BY Tx_DtTm
    """, 
    { "sit_id": sit_id })

dose_hst_datetimes = [dose_hst[0].strftime('%Y-%m-%d %H:%M') 
                    for dose_hst in dose_hsts]
print(f"Dose_Hst records for site {sites[0][2]}:")
pprint.pprint(dose_hst_datetimes[:10])

## Cluster Dose_Hst in to sessions
Mosaiq's data schema doesn't explicitly group Dose_Hst and Offset records in to sessions/fractions, but a simple clustering trick is generally enough to form the sessions.

The ```cluster_sessions``` function uses a [hierarchical clustering algorithm](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.AgglomerativeClustering.html) to cluster the Dose_Hst records.  

To demonstrate the clustering, generate a list of irregularly spaced datetimes to pass to ```cluster_sessions```, which will return a tuple for each session of:
* session number (from 1)
* session start date/time
* session end date/time

In [ ]:
from session_offsets_calculator import cluster_sessions
from datetime import datetime, timedelta

test_datetimes = [datetime.now() + timedelta(hours=h*5 + j) 
                    for h in range(3) for j in range(3)]

print('Mock tx date/times:')
for datetime in test_datetimes: 
    print('\t', str(datetime))

print(f"\nClustered in to sessions with {3} hour inverval:")
for session in cluster_sessions(test_datetimes, interval=timedelta(hours=3)):
    print(f"\tSession#{session[0]}: {str(session[1])} to {str(session[2])}")

The sessions for the queried site can now be created using the ```sessions_for_site``` function, which first queries for Dose_Hst records, and then calls ```cluster_sessions``` on the Dose_Hst.Tx_DtTm.

In [ ]:
from session_offsets_calculator import sessions_for_site

print(f"Dose_Hst.Tx_DtTm-based session intervals "
      f"for SIT_SET_ID = {sit_set_id} in Msq db:")
for session in sessions_for_site(connection, sit_set_id):
    print(f"\tSession#{session[0]}: {str(session[1])} to {str(session[2])}")

## Session Offsets
Now that we can get the sessions for a site, we can also query to find any Offsets that occur within a +/- 1 hour time window of the session interval.  This is done by the ```session_offsets_for_site``` function.

Session offsets are returned by a generator as a tuple of Session Number and Offset values.  If no Offset falls within the window for a session, then None is returned for the offset member of the tuple.

In [ ]:
from session_offsets_calculator import session_offsets_for_site

print(f"Offset records for SIT_SET_ID {sit_set_id}")    
for session_num, offset in session_offsets_for_site(
    connection, sit_set_id, interval=timedelta(hours=1)
):
    if offset:
        print(f"\tSession#{session_num}: "
            f"{offset[0].strftime('%Y-%m-%d %H:%M')}: "
            f"{offset[1]}/{offset[2]}/{offset[3]}")
    else:
        print(f"\tSession#{session_num}: no session offsets")